# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [2]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [26]:
def num_of_crossing_edges(arr, graph):
    counter = 0
    for it in graph[1]:
        if (arr[it[0] - 1] != arr[it[1] - 1]):
            counter += 1
    return counter 

def check_ans(division, graph):
    ans = num_of_crossing_edges(division, graph)
    print("Answer:{}".format(ans))
    for i in range(10):
        np.random.shuffle(division)
        rand = num_of_crossing_edges(division, graph)
        print("Random answer:{}".format(rand))

In [4]:
import numpy as np
from itertools import permutations


In [41]:
def find_better(graph, division):
    changed = np.zeros_like(division)
    found_better = False
    minim = num_of_crossing_edges(division, graph)
    for i in range(len(division)):
        for j in range(i, len(division)):
            if (division[i] != division[j]):      
                division[i] = 1 - division[i]
                division[j] = 1 - division[j]
                temp = num_of_crossing_edges(division, graph)
                if (temp < minim):
                    minim = temp
                    changed = np.copy(division)
                    found_better = True
                division[i] = 1 - division[i]
                division[j] = 1 - division[j]
    return (found_better, changed)

In [9]:
def best_from_others(graph, division, not_equal_to):
    changed = np.zeros_like(division)
    first = True
    for i in range(len(division)):
        for j in range(i, len(division)):
            if (division[i] != division[j]):        
                division[i] = 1 - division[i]
                division[j] = 1 - division[j]
                if (not(np.array_equal(division, not_equal_to))):
                    temp = num_of_crossing_edges(division, graph)
                    if (first):
                        minim = temp
                        changed = np.copy(division)
                        first = False
                    elif (temp < minim):
                        minim = temp
                        changed = np.copy(division)
                        first = False
                division[i] = 1 - division[i]
                division[j] = 1 - division[j]
    return changed

In [48]:
DEPTH = 1
def variable_depth_local_search(graph):
    # По сути, в starting_point хранится разбиение, в котором первые 
    # [n/2] + 1 вершин по номеру лежат в первой доле, оставшиеся - 
    # во второй. Помня об этом, изменим формат хранения разбиения, 
    # чтобы было удобнее пользоваться кодом предыдущей задачи.
    #starting_point = set(range(1, len(graph[0]) // 2 + 1))
    division = np.zeros(len(graph[0]), dtype=int)
    division[:int(len(graph[0]) / 2)] = 1
    not_stop = True
    changed = division    
    step = np.zeros(len(division))
    counter = 0
    prev_min = 0
    prev_division = np.copy(division)
    first_in_chain = True
    
    while (not_stop):
        res = find_better(graph, division)
        not_stop = False
        if (res[0]):
            division = res[1]
            not_stop = True
            counter = 0
            prev_min = num_of_crossing_edges(division, graph)
            first_in_chain = True
        else:
            if (first_in_chain):
                prev_division = np.copy(division)
            not_equal_to = np.copy(division)
            division = best_from_others(graph, division, not_equal_to)
            counter += 1
            if (counter >= DEPTH):
                if (num_of_crossing_edges(division, graph) >= num_of_crossing_edges(prev_division, graph)):
                    check_ans(division, graph)
                    return prev_division
    check_ans(division, graph)
    return division

In [110]:
import time

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    for filename in filenames:
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

In [112]:
run_all()

Solving instance add20.graph…

KeyboardInterrupt: 

In [49]:
gr = ((1, 2, 3, 4, 5, 6, 7, 8, 9), ((6, 8),  (1, 4), (2, 3), (3, 4), (2, 5), (6, 7), (7, 9), (8, 9), (1, 5), (1, 7), (1, 9)))
print(variable_depth_local_search(gr))

Answer:4
Random answer:5
Random answer:7
Random answer:4
Random answer:9
Random answer:5
Random answer:5
Random answer:5
Random answer:7
Random answer:6
Random answer:8
[0 1 1 1 1 0 0 0 0]


## Выводы
На моем компьютере даже с глубиной поиска 1 ответ за разумное время не высчитывается. Выше приведен пример маленького теста, на нем алгоритм работает.